In [ ]:
# initial setup
try:
    # settings colab:
    import google.colab
        
except ModuleNotFoundError:    
    # settings local:
    args = "conda-forge"
    %run "../../../common/0_notebooks_base_setup.py" {args}

---

<img src='../../../common/logo_DH.png' align='left' width=35%/>


## Checkpoint - Desbalance de clases + Feature Selection

En este checkpoint trabajaremos con el dataset de préstamos crediticios 'loans.csv'.

En la primera parte combinaremos los elementos vistos en la práctica guiada de clases desbalanceadas para resamplear el data set de entrenamiento.

Luego implementaremos un modelo de regresión logística combinado con feature selection por eliminación recursiva.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression,LogisticRegressionCV
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTENC
from sklearn.metrics import accuracy_score,plot_confusion_matrix,roc_auc_score, classification_report, confusion_matrix, precision_recall_curve, auc


In [ ]:
loans = pd.read_csv('../Data/loans.csv',low_memory=False)
loans.dropna(inplace=True)
loans.reset_index(drop=True,inplace=True)
loans['bad_loans'].value_counts(normalize=True)

In [ ]:
loans.sample(3)

### 1. Train test split 

Hagamos un train test split estratificado por la variable target (bad loans). 

¿Cuál es el desbalance de clases?

¿Cuál es el ratio clase mayoritaria/minoritaria?

In [ ]:
# Train test split
X_train,X_test,y_train,y_test=train_test_split(loans.drop(['bad_loans'],axis=1),\
                                               loans['bad_loans'],stratify=loans['bad_loans'],random_state=0)

print('Tamaño del training set:')
print(X_train.shape)

print('\nBalance de clases:')
print(y_train.value_counts(normalize=False))


print('\nBalance de clases norm:')
print(y_train.value_counts(normalize=True))

print('\nRatio entre las clases:')
print((y_train==1).sum()/(y_train==0).sum())




### 2. SMOTE-NC 

Implementemos la herramienta SMOTENC para resamplear la clase minoritaria en el dataset de entrenamiento. Para ello debemos identificar previamente las variables categóricas y pasarlas como argumento al instanciar el objeto SMOTENC.

Usemos un sampling_strategy=0.5, de modo que la clase mayoritaria tenga el doble de observaciones que la minoritaria.

¿Cuál es el balance de clases ahora?

¿Cuál es el ratio mayoritaria/minoritaria?

In [ ]:
categorical_cols=(X_train.dtypes=='object').values
numerical_cols= ~categorical_cols

sm=SMOTENC(categorical_features=categorical_cols ,sampling_strategy=0.5)
X_train_os,y_train_os=sm.fit_resample(X_train,y_train)

In [ ]:
print('Balance de clases:')
print(y_train_os.value_counts(normalize=False))

print('\nBalance de clases norm:')
print(y_train_os.value_counts(normalize=True))

print('\nRatio entre las clases:')
print((y_train_os==1).sum()/(y_train_os==0).sum())

print('\nTamaño de Xtrain:')
print(X_train_os.shape)

### 3. Undersampling

A partir del dataset aumentado en el paso anterior balanceemos las clases sampleando aleatoriamente observaciones de la clase mayoritaria. Para esto usemos un RandomUnderSampler con sampling_strategy='majority' de modo que queden las clases balanceadas.

¿Cómo quedó el balance de clases?

¿Y el tamaño del dataset de entrenamiento?


In [ ]:
undersampler=RandomUnderSampler(sampling_strategy='majority');
X_train_us,y_train_us=undersampler.fit_resample(X_train_os,y_train_os)

print('Balance de clases:')
print(y_train_us.value_counts(normalize=False))

print('\nBalance de clases norm:')
print(y_train_us.value_counts(normalize=True))

print('\nRatio entre las clases:')
print((y_train_us==1).sum()/(y_train_us==0).sum())

print('\nTamaño de Xtrain:')
print(X_train_us.shape)

### 4. Variables dummy

Utilicemos la herramienta OneHotEncoder para llevar a dummies las variables categoricas del training set. Luego transformamos de manera conforme las categóricas del test set.

In [ ]:
# dummy variables

enc=OneHotEncoder(drop='first',sparse=False)
train_dummies=enc.fit_transform(X_train_us.loc[:,categorical_cols])
train_dummies=pd.DataFrame(train_dummies)
train_dummies.columns=[x for cat_list in enc.categories_ for x in cat_list[1:]]

X_train_final=X_train_us.loc[:,numerical_cols].join(train_dummies)

test_dummies=enc.transform(X_test.loc[:,categorical_cols])
test_dummies=pd.DataFrame(test_dummies)
test_dummies.columns=[x for cat_list in enc.categories_ for x in cat_list[1:]]
test_dummies.index=X_test.index

X_test_final=X_test.loc[:,numerical_cols].join(test_dummies)

y_train_final=y_train_us # solo por nomenclatura
y_test_final=y_test # solo por nomenclatura

print(X_train_final.shape)
print(X_test_final.shape)

### 5. Regresión logística

Implementemos una regresión logística usando LogisticRegressionCV. 

¿Qué medida de scoring conviene usar?

Veamos las métricas principales de performance en el test set: 

* matriz de confusión
* clasification report
* área bajo la curva ROC
* área bajo la curva Precision-Recall

In [ ]:
def logistic_regression_report(X_train,X_test,y_train,y_test):
    
    scaler=StandardScaler()    
        
    X_train_sc=scaler.fit_transform(X_train)
    X_test_sc=scaler.transform(X_test)
        
    model=LogisticRegressionCV(scoring='f1')
        
    model.fit(X_train_sc,y_train)
    y_pred=model.predict(X_test_sc)
    y_proba=model.predict_proba(X_test_sc)
    
    print(classification_report(y_test,y_pred))
    
    print('Area bajo la curva ROC:',np.round(roc_auc_score(y_test,y_proba[:,1]),4))
    
    precision, recall,threshold=precision_recall_curve(y_test,y_proba[:,1]);

    print('Area bajo la curva Precision-Recall:',np.round(auc(recall,precision),4))

    plot_confusion_matrix(model,X_test_sc,y_test,cmap='Blues');
    
    return model
    

In [ ]:
m=logistic_regression_report(X_train_final,X_test_final,y_train_final,y_test_final)

### 6. Feature selection

Utilicen la herramientsa RFECV combinada con una regresión logística para seleccionar el mejor subset de features. Usen una regresión logística con el hiperparámetro de regularización C ajustado en el punto anterior. 

Lo correcto sería optimizar este parámetro para cada subset de features, es decir hacer dos validaciones cruzadas anidadas (una para RFE y otra para la regresión logística), pero por 
economía de cómputo lo haremos con el valor de C fijo.

¿Cuáles features fueron seleccionadas?

¿Cuáles fueron descartadas?

¿Cuál es la performance del modelo?

In [ ]:
from sklearn.feature_selection import RFECV
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=5, shuffle = True)

model = LogisticRegression(C=m.C_[0]) # La penalización óptima podría depender del nro de Features, pero para no sobrecargar el cómputo no optimizaremos C para cada representación de los datos

scaler=StandardScaler()    
X_train_sc=scaler.fit_transform(X_train_final)
X_test_sc=scaler.transform(X_test_final)


selector = RFECV(model, scoring='f1',step = 1, cv=skf, verbose=0,n_jobs=3)
selector.fit(X_train_sc, y_train_final)

In [ ]:
scaler.fit_transform(X_train_final).shape

In [ ]:
X_train_reduced=X_train_sc[:,selector.support_]
X_test_reduced=X_test_sc[:,selector.support_]
logistic_regression_report(X_train_reduced,X_test_reduced,y_train_final,y_test_final)

In [ ]:
print('Features Conservadas:')
print(X_train_final.columns[selector.support_])

print('\nFeatures Descartadas')
print(X_train_final.columns[~selector.support_])